<a href="https://colab.research.google.com/github/hassanalubeidi/RAG/blob/main/RAG_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install chromadb
!pip install openai

In [ ]:
from chromadb.utils import embedding_functions
import chromadb
from openai import OpenAI

In [ ]:
from google.colab import userdata
openai_key=userdata.get('openai-key')

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=openai_key, model_name="text-embedding-3-small"
)
# Initialize the Chroma client with persistence
chroma_client = chromadb.PersistentClient(path="chroma_persistent_storage")
collection_name = "document_qa_collection"
collection = chroma_client.get_or_create_collection(
    name=collection_name, embedding_function=openai_ef
)



In [ ]:
client = OpenAI(api_key=openai_key)

resp = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "What is human life expectancy in the United States?",
        },
    ],
)

resp

ChatCompletion(id='chatcmpl-AUtfdBecUPZ30HeJiZapgy91fz7F0', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="As of the most recent data up to 2023, the average life expectancy in the United States is approximately 76 years. It's important to note that life expectancy can vary based on factors such as gender, ethnicity, and socioeconomic status. Additionally, the COVID-19 pandemic has impacted life expectancy trends in recent years.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1731927201, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_45cf54deae', usage=CompletionUsage(completion_tokens=64, prompt_tokens=27, total_tokens=91, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=

In [ ]:
# prompt: Download all the .txt files from https://github.com/pdichone/rag-intro-chat-with-docs/tree/main/news_articles and place them in ./news_articles

import os
import requests
from urllib.parse import urljoin

def download_txt_files(base_url, target_dir):
    """Downloads all .txt files from a given URL to a target directory."""

    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    response = requests.get(base_url)
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)

    # Find all .txt file links
    txt_file_links = []
    for line in response.text.splitlines():
      if ".txt" in line:
        start = line.find('href="') + 6
        end = line.find('"', start)
        if start > 5 and end > start:
          txt_file_links.append(line[start:end])

    for link in txt_file_links:
        file_url = urljoin(base_url, link)
        filename = os.path.basename(file_url)  # Extract filename
        filepath = os.path.join(target_dir, filename)

        try:
            file_response = requests.get(file_url, stream=True)
            file_response.raise_for_status()
            with open(filepath, "wb") as f:
                for chunk in file_response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"Downloaded: {filename}")
        except requests.exceptions.RequestException as e:
            print(f"Error downloading {filename}: {e}")

# Example usage
base_url = "https://github.com/pdichone/rag-intro-chat-with-docs/tree/main/news_articles"
target_dir = "./news_articles"

download_txt_files(base_url, target_dir)

Downloaded: main


In [ ]:
import os
# Function to load documents from a directory
def load_documents_from_directory(directory_path):
    print("==== Loading documents from directory ====")
    documents = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):
            with open(
                os.path.join(directory_path, filename), "r", encoding="utf-8"
            ) as file:
                documents.append({"id": filename, "text": file.read()})
    return documents

# Load documents from the directory
directory_path = "./news_articles"
documents = load_documents_from_directory(directory_path)
print(f"Loaded {len(documents)} documents")


==== Loading documents from directory ====
Loaded 0 documents


In [ ]:
# Function to split text into chunks
def split_text(text, chunk_size=1000, chunk_overlap=20):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - chunk_overlap
    return chunks

# Split documents into chunks
chunked_documents = []
for doc in documents:
    chunks = split_text(doc["text"])
    print("==== Splitting docs into chunks ====")
    for i, chunk in enumerate(chunks):
        chunked_documents.append({"id": f"{doc['id']}_chunk{i+1}", "text": chunk})

print(f"Split documents into {len(chunked_documents)} chunks")

==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
Split documents into 184 chunks


In [ ]:
# Function to generate embeddings using OpenAI API
def get_openai_embedding(text):
    response = client.embeddings.create(input=text, model="text-embedding-3-small")
    embedding = response.data[0].embedding
    print("==== Generating embeddings... ====")
    return embedding


# Generate embeddings for the document chunks
for doc in chunked_documents:
    print("==== Generating embeddings... ====")
    doc["embedding"] = get_openai_embedding(doc["text"])

print(doc["embedding"])

==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embeddings... ====
==== Generating embe

In [ ]:
# Upsert documents with embeddings into Chroma
for doc in chunked_documents:
    print("==== Inserting chunks into db;;; ====")
    collection.upsert(
        ids=[doc["id"]], documents=[doc["text"]], embeddings=[doc["embedding"]]
    )

==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserting chunks into db;;; ====
==== Inserti

In [ ]:
# Function to query documents
def query_documents(question, n_results=2):
    # query_embedding = get_openai_embedding(question)
    results = collection.query(query_texts=question, n_results=n_results)

    # Extract the relevant chunks
    relevant_chunks = [doc for sublist in results["documents"] for doc in sublist]
    print("==== Returning relevant chunks ====")
    return relevant_chunks
    # for idx, document in enumerate(results["documents"][0]):
    #     doc_id = results["ids"][0][idx]
    #     distance = results["distances"][0][idx]
    #     print(f"Found document chunk: {document} (ID: {doc_id}, Distance: {distance})")


# Function to generate a response from OpenAI
def generate_response(question, relevant_chunks):
    context = "\n\n".join(relevant_chunks)
    prompt = (
        "You are an assistant for question-answering tasks. Use the following pieces of "
        "retrieved context to answer the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the answer concise."
        "\n\nContext:\n" + context + "\n\nQuestion:\n" + question
    )

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": prompt,
            },
            {
                "role": "user",
                "content": question,
            },
        ],
    )

    answer = response.choices[0].message
    return answer


# Example query
# query_documents("tell me about AI replacing TV writers strike.")
# Example query and response generation
question = "tell me about databricks"
relevant_chunks = query_documents(question)
answer = generate_response(question, relevant_chunks)

print(answer)

==== Returning relevant chunks ====
ChatCompletionMessage(content="Databricks is a data and AI company known for its data analytics platform that helps organizations manage and analyze large volumes of data efficiently. Recently, they acquired Okera to enhance their data governance capabilities, integrating Okera's AI-powered systems into their existing Unity Catalog. This acquisition aims to provide advanced access controls and governance solutions, addressing the complexities of managing data across multiple clouds.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [ ]:
question = "tell me about slack"
relevant_chunks = query_documents(question)
answer = generate_response(question, relevant_chunks)

print(answer)

==== Returning relevant chunks ====
ChatCompletionMessage(content='Slack is a communications platform that has evolved to integrate directly with enterprise applications, reducing the need for task switching. Recently, it announced enhancements that will incorporate AI, including SlackGPT, which offers AI-driven experiences and tools like conversation summaries and writing assistance. The platform aims to leverage the existing institutional knowledge within Slack to improve workflows and communications.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
